Project: /meridian/_project.yaml
Book: /meridian/_book.yaml

<style>
devsite-code .tfo-notebook-code-cell-output {
  max-height: 300px;
  overflow: auto;
  background: rgba(255, 247, 237, 1);  /* orange bg to distinguish from input code cells */
}

devsite-code .tfo-notebook-code-cell-output + .devsite-code-buttons-container button {
  background: rgba(255, 247, 237, .7);  /* orange bg to distinguish from input code cells */
}

devsite-code[dark-code] .tfo-notebook-code-cell-output {
  background: rgba(64, 78, 103, 1);  /* medium slate */
}

devsite-code[dark-code] .tfo-notebook-code-cell-output + .devsite-code-buttons-container button {
  background: rgba(64, 78, 103, .7);  /* medium slate */
}

/* override default table styles for notebook buttons */
.devsite-table-wrapper .tfo-notebook-buttons {
  display: inline-block;
  margin-left: 3px;
  width: auto;
}

.tfo-notebook-buttons tr {
  background: 0;
  border: 0;
}

.tfo-notebook-buttons td {
  padding-left: 0;
  padding-right: 20px;
}

.tfo-notebook-buttons {
  --tfo-notebook-buttons-box-shadow: 0 1px 2px 0 rgba(60, 64, 67, .3), 0 1px 3px 1px rgba(60, 64, 67, .15);
}

.tfo-notebook-buttons a,
.tfo-notebook-buttons :link,
.tfo-notebook-buttons :visited {
  border-radius: 8px;
  box-shadow: var(--tfo-notebook-buttons-box-shadow);
  color: #202124;
  padding: 12px 24px;
  transition: box-shadow 0.2s;
}

.tfo-notebook-buttons a:hover,
.tfo-notebook-buttons a:focus {
  box-shadow: var(--tfo-notebook-buttons-box-shadow);
}

.tfo-notebook-buttons td > a {
  -webkit-box-align: center;
  -ms-flex-align: center;
  align-items: center;
  display: -webkit-box;
  display: -ms-flexbox;
  display: flex;
}

.tfo-notebook-buttons td > a > img {
  margin-right: 8px;
}
</style>

<table class="tfo-notebook-buttons tfo-api nocontent" align="left">
  <tbody>
    <tr>
      <td>
        <a target="_blank" href="https://colab.research.google.com/github/google/meridian/blob/main/demo/Meridian_Getting_Started.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
      </td>
      <td>
        <a target="_blank" href="https://github.com/google/meridian/blob/main/demo/Meridian_Getting_Started.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
      </td>
    </tr>
  </tbody>
</table>

# **Introduction to Meridian Demo**

Welcome to the Meridian end-to-end demo. This simplified demo showcases the fundamental functionalities and basic usage of the library, including working examples of the major modeling steps:


<ol start="0">
  <li><a href="#install">Install and Enviroment Configuration</a></li>
  <li><a href="#load-data">Load the data</a></li>
  <li><a href="#configure-model">Configure the model</a></li>
  <li><a href="#quality-checks">Run post-modeling quality checks</a></li>
  <li><a href="#model-diagnostics">Run model diagnostics</a></li>
  <li><a href="#generate-summary">Generate model results & two-page output</a></li>
  <li><a href="#generate-optimize">Run budget optimization & two-page output</a></li>
  <li><a href="#save-model">Save the model object</a></li>
  <li><a href="#scenario-planning">Interactive Scenario Planning</a></li>
</ol>


Note that this notebook skips all of the exploratory data analysis and preprocessing steps. It assumes that you have completed these tasks before reaching this point in the demo.

This notebook utilizes sample data. As a result, the numbers and results obtained might not accurately reflect what you encounter when working with a real dataset.

<a name="install"></a>
## Step 0: Install and Enviroment Configuration

1\. Make sure you are using one of the available GPU Colab runtimes which is **required** to run Meridian. You can change your notebook's runtime in `Runtime > Change runtime type` in the menu. All users can use the T4 GPU runtime which is sufficient to run the demo colab, free of charge. Users who have purchased one of Colab's paid plans have access to premium GPUs (such as V100, A100 or L4 Nvidia GPU).

2\. Install the latest version of Meridian, and verify that GPU is available.

In [3]:
# Install meridian: from PyPI @ latest release
!pip install --upgrade google-meridian[colab,and-cuda,schema]

# Install meridian: from PyPI @ specific version
# !pip install google-meridian[colab,and-cuda,schema]==1.3.1

# Install meridian: from GitHub @HEAD
# !pip install --upgrade "google-meridian[colab,and-cuda,schema] @ git+https://github.com/google/meridian.git@main"

In [4]:
import IPython
from meridian import constants
from meridian.analysis import analyzer
from meridian.analysis import optimizer
from meridian.analysis import summarizer
from meridian.analysis import visualizer
from meridian.analysis.review import reviewer
from meridian.data import data_frame_input_data_builder
from meridian.model import model
from meridian.model import prior_distribution
from meridian.model import spec
from schema.serde import meridian_serde
import numpy as np
import pandas as pd
# check if GPU is available
from psutil import virtual_memory
import tensorflow as tf
import tensorflow_probability as tfp

ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))
print(
    'Num GPUs Available: ',
    len(tf.config.experimental.list_physical_devices('GPU')),
)
print(
    'Num CPUs Available: ',
    len(tf.config.experimental.list_physical_devices('CPU')),
)

Your runtime has 13.6 gigabytes of available RAM

Num GPUs Available:  1
Num CPUs Available:  1


3\. Mount a storage. Use `meridian_root` to refer the mounted root. The mounted root will be used to <a href="#save-model">save trained model</a>, <a href="#generate-summary">stage two-pager output</a> and <a href="#scenario-planning">generate scenario planning dashboard</a>.

For Colab Free/Pro user, we will use the `MyDrive` folder in Google Drive as the external storage. For Colab Enterprise user, we will use <a href="https://docs.cloud.google.com/storage/docs/cloud-storage-fuse/overview">Cloud FUSE</a> to mount a GCS bucket.

In [6]:
# @markdown If you are using Colab Free, Colab Pro, run this cell to mount your Google Drive.
from google.colab import drive
drive_mount = '/content/drive'
drive.mount(drive_mount, force_remount=True)
subfolder = '' # @param {"type":"string","placeholder": "Optional, specifying a subfolder is recommended for organizing distinct execution runs."}
# Change this "MyDrive" to other share folders name if you would like to use a different drive.
meridian_root = f'{drive_mount}/MyDrive/{subfolder}'
is_enterprise_user=False

Mounted at /content/drive


In [ ]:
import os

# @markdown If you are using Colab Enterprise, run this cell to mount a GCS bucket.
project_id = ""# @param {"type":"string","placeholder": "Cloud project id"}
bucket_name = "" # @param {"type":"string","placeholder": "GCS bucket that contains your model"}
subfolder = "" # @param {"type":"string","placeholder": "Optional, specifying a subfolder is recommended for organizing distinct execution runs."}
os.environ['GOOGLE_CLOUD_PROJECT'] = project_id
!gcloud config set project {project_id}
!gcloud auth login
!mkdir /content/{bucket_name}

# Uncomment below if you don't have gcsfuse installed
#!echo "deb https://packages.cloud.google.com/apt gcsfuse-`lsb_release -c -s` main" | sudo tee /etc/apt/sources.list.d/gcsfuse.list
#!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
#!apt-get update
#!apt-get install gcsfuse

!gcsfuse --implicit-dirs {bucket_name} /content/{bucket_name}
meridian_root = f'/content/{bucket_name}/{subfolder}'
is_enterprise_user=True
#!fusermount -u /content/{bucket_name}

<a name="load-data"></a>
## Step 1: Load the data

Load the [simulated dataset in CSV format](https://github.com/google/meridian/blob/main/meridian/data/simulated_data/csv/geo_all_channels.csv) as follows.

1\. Read the data into a Pandas DataFrame.

In [7]:
df = pd.read_csv(
    # Optionally, use `f"${meridian_root}/<path_to_csv>"` to load data from the mounted storage.
    "https://raw.githubusercontent.com/google/meridian/refs/heads/main/meridian/data/simulated_data/csv/geo_all_channels.csv"
)

2\. Create a DataFrameInputDataBuilder instance.

In [8]:
builder = data_frame_input_data_builder.DataFrameInputDataBuilder(
    kpi_type='non_revenue',
    default_kpi_column='conversions',
    default_revenue_per_kpi_column='revenue_per_conversion',
)

3\. Offer the components to the builder. Note that the components may be offered all at once or piecewise.

In [9]:
builder = (
    builder.with_kpi(df)
    .with_revenue_per_kpi(df)
    .with_population(df)
    .with_controls(
        df, control_cols=["sentiment_score_control", "competitor_sales_control"]
    )
)

channels = ["Channel0", "Channel1", "Channel2", "Channel3", "Channel4"]
builder = builder.with_media(
    df,
    media_cols=[f"{channel}_impression" for channel in channels],
    media_spend_cols=[f"{channel}_spend" for channel in channels],
    media_channels=channels,
)

4. If your data includes organic media or non-media treatments, you can add them using `with_organic_media` and `with_non_media_treatments` methods. For the definition of each variable, see
[Collect and organize your data](https://developers.google.com/meridian/docs/pre-modeling/collect-data)

In [10]:
builder = builder.with_non_media_treatments(
    df, non_media_treatment_cols=['Promo']
).with_organic_media(
    df,
    organic_media_cols=['Organic_channel0_impression'],
    organic_media_channels=['Organic_channel0'],
)

5. Finally, build the InputData.

In [11]:
data = builder.build()

Note that the simulated data here does not contain reach and frequency. We recommend including reach and frequency data whenever they are available. For information about the advantages of utilizing reach and frequency, see [Bayesian Hierarchical Media Mix Model Incorporating Reach and Frequency Data](https://research.google/pubs/bayesian-hierarchical-media-mix-model-incorporating-reach-and-frequency-data/#:~:text=By%20incorporating%20R%26F%20into%20MMM,based%20on%20optimal%20frequency%20recommendations.). For code snippet for loading reach and frequency data, see [Load geo-level data with reach and frequency](https://developers.google.com/meridian/docs/user-guide/load-geo-data-with-rf)

The documentation provides guidance for instances where reach and frequency data is accessible for specific channels. Additionally, for information about how to load other data types and formats, including data with reach and frequency, see [Supported data types and formats](https://developers.google.com/meridian/docs/user-guide/supported-data-types-formats).

<a name="configure-model"></a>
## Step 2: Configure the model

Meridian uses Bayesian framework and Markov Chain Monte Carlo (MCMC) algorithms to sample from the posterior distribution.

1\. Inititalize the `Meridian` class by passing the loaded data and the customized model specification. One advantage of Meridian lies in its capacity to calibrate the model directly through ROI priors, as described in [Media Mix Model Calibration With Bayesian Priors](https://research.google/pubs/media-mix-model-calibration-with-bayesian-priors/). In this particular example, the ROI priors for all media channels are identical, with each being represented as Lognormal(0.2, 0.9).

In [12]:
roi_mu = 0.2  # Mu for ROI prior for each media channel.
roi_sigma = 0.9  # Sigma for ROI prior for each media channel.
prior = prior_distribution.PriorDistribution(
    roi_m=tfp.distributions.LogNormal(roi_mu, roi_sigma, name=constants.ROI_M)
)
model_spec = spec.ModelSpec(prior=prior, enable_aks=True)

mmm = model.Meridian(input_data=data, model_spec=model_spec)

2\. Use the `sample_prior()` and `sample_posterior()` methods to obtain samples from the prior and posterior distributions of model parameters. If you are using the T4 GPU runtime this step may take about 10 minutes for the provided data set.

In [13]:
%%time
mmm.sample_prior(500)
mmm.sample_posterior(
    n_chains=10, n_adapt=2000, n_burnin=500, n_keep=1000, seed=0
)

/usr/local/lib/python3.12/dist-packages/meridian/model/knots.py:505: RuntimeWarning: overflow encountered in cast
  np.float32(math.comb(ncol, design_mat.shape[1]))


CPU times: user 20min 42s, sys: 23.6 s, total: 21min 5s
Wall time: 21min 3s


For more information about configuring the parameters and using a customized model specification, such as setting different ROI priors for each media channel, see [Configure the model](https://developers.google.com/meridian/docs/user-guide/configure-model).

<a name="quality-checks"></a>
## Step 3: Run post-modeling quality checks

These post-modeling quality checks are designed to diagnose common issues related to model convergence, specification, and plausibility. Run the following command to generate the results for all necessary diagnostics:

In [20]:
reviewer.ModelReviewer(mmm).run()

Model Quality Checks
Overall Status: PASS
Summary: Passed: No major quality issues were identified.

Check Results:
----------------------------------------
Convergence Check:
  Status: PASS
  Recommendation: The model has likely converged, as all parameters have R-hat values < 1.2.
----------------------------------------
Baseline Check:
  Status: PASS
  Recommendation: The posterior probability that the baseline is negative is 0.00. We recommend visually inspecting the baseline time series in the Model Fit charts to confirm this.
----------------------------------------
BayesianPPP Check:
  Status: PASS
  Recommendation: The Bayesian posterior predictive p-value is 0.99. The observed total outcome is consistent with the model's posterior predictive distribution.
----------------------------------------
GoodnessOfFit Check:
  Status: PASS
  Recommendation: R-squared = 0.7738, MAPE = 0.2557, and wMAPE = 0.1998. These goodness-of-fit metrics are intended for guidance and relative compar

<a name="model-diagnostics"></a>
## Step 4: Run model diagnostics

To further assess convergence and model fit, you can use the methods from `visualizer` module.

1\. Assess convergence. Run the following code to generate r-hat statistics. R-hat close to 1.0 indicate convergence. R-hat < 1.2 indicates approximate convergence and is a reasonable threshold for many problems.

In [21]:
model_diagnostics = visualizer.ModelDiagnostics(mmm)
model_diagnostics.plot_rhat_boxplot()

alt.LayerChart(...)

2\. Assess the model's fit by comparing the expected sales against the actual sales.

In [22]:
model_fit = visualizer.ModelFit(mmm)
model_fit.plot_model_fit()

alt.LayerChart(...)

For more information and additional model diagnostics checks, see [Modeling diagnostics](https://developers.google.com/meridian/docs/user-guide/model-diagnostics).

<a name="generate-summary"></a>
## Step 5: Generate model results & two-page output

To export the two-page HTML summary output, initialize the `Summarizer` class with the model object. Then pass in the filename, filepath, start date, and end date to `output_model_results_summary` to run the summary for that time duration and save it to the specified file.

In [23]:
mmm_summarizer = summarizer.Summarizer(mmm)

In [24]:
filepath = meridian_root
start_date = '2021-01-25'
end_date = '2024-01-15'
mmm_summarizer.output_model_results_summary(
    'summary_output.html', filepath, start_date, end_date
)

/usr/local/lib/python3.12/dist-packages/numpy/lib/_function_base_impl.py:4779: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
/usr/local/lib/python3.12/dist-packages/meridian/analysis/analyzer.py:3348: UserWarning: Effectiveness is not reported because it does not have a clear interpretation by time period.
  warnings.warn(


Here is a preview of the two-page output based on the simulated data:

In [25]:
IPython.display.HTML(filename=f'{meridian_root}/summary_output.html')

Dataset,R-squared,MAPE,wMAPE
All Data,0.77,3%,3%


For a customized two-page report, model results summary table, and individual visualizations, see [Model results report](https://developers.google.com/meridian/docs/user-guide/generate-model-results-report) and [plot media visualizations](https://developers.google.com/meridian/docs/user-guide/plot-media-visualizations).





<a name="generate-optimize"></a>
## Step 6: Run budget optimization & generate an optimization report

You can choose what scenario to run for the budget allocation. In default scenario, you find the optimal allocation across channels for a given budget to maximize the return on investment (ROI).

Alternatively, if you would like to have a sharable interactive dashboard, check out [Meridian Scenario Planner](https://developers.google.com/meridian/docs/scenario-planning/meridian-scenario-planner).

1\. Instantiate the `BudgetOptimizer` class and run the `optimize()` method without any customization, to run the default library's Fixed Budget Scenario to maximize ROI.

In [26]:
%%time
budget_optimizer = optimizer.BudgetOptimizer(mmm)
optimization_results = budget_optimizer.optimize()

CPU times: user 41.7 s, sys: 1.72 s, total: 43.4 s
Wall time: 49.1 s


2\. Export the 2-page HTML optimization report, which contains optimized spend allocations and ROI.

In [27]:
filepath = meridian_root
optimization_results.output_optimization_summary(
    'optimization_output.html', filepath
)

In [28]:
IPython.display.HTML(filename=f'{meridian_root}/optimization_output.html')

Channel,Non-optimized spend,Optimized spend
Channel4,22%,28%
Channel3,40%,28%
Channel0,18%,24%
Channel1,14%,13%
Channel2,6%,7%


For information about customized optimization scenarios, such as flexible budget scenarios, see [Budget optimization scenarios](https://developers.google.com/meridian/docs/user-guide/budget-optimization-scenarios). For more information about optimization results summary and individual visualizations, see [optimization results output](https://developers.google.com/meridian/docs/user-guide/generate-optimization-results-output) and [optimization visualizations](https://developers.google.com/meridian/docs/user-guide/plot-optimization-visualizations).


Optimization can also be performed on a hypothetical data representing a future scenario. The new data takes the same structure as the input data and encodes an anticipated flighting pattern, cost per media unit, and revenue per kpi.

3\. Load the [simulated dataset in CSV format](https://github.com/google/meridian/blob/main/meridian/data/simulated_data/csv/hypothetical_geo_all_channels.csv) into Pandas DataFrame.

In [29]:
df = pd.read_csv(
    "https://raw.githubusercontent.com/google/meridian/refs/heads/main/meridian/data/simulated_data/csv/hypothetical_geo_all_channels.csv"
)



4\. New data is read from a csv file and converted into a set of multi-dimensional arrays. The arrays are used to construct a `DataTensors` instance, which is passed to `optimize()` as the `new_data` argument.

Constructing a `DataTensors` instance requires that all arrays have "time" and "geo" dimensions. Alternatively, the `BudgetOptimizer.create_optimization_tensors` method can be used to construct a `DataTensors` instance. This helper method can simplify the process, particularly when you do not need "time" and "geo" dimensions for all inputs. For example, it can be convenient if you want to assume a constant "revenue per kpi" or "cost per media unit" for all geos and time periods.

In [ ]:
n_geos = mmm.n_geos
n_media_channels = mmm.n_media_channels
n_non_media_channels = mmm.n_non_media_channels
n_organic_media_channels = mmm.n_organic_media_channels

# The number of time periods and time range do not need to match the input data.
df[constants.TIME] = pd.to_datetime(df[constants.TIME], errors='coerce')
unique_times = sorted(df[constants.TIME].unique())
n_times = len(unique_times)

geos = mmm.input_data.geo.values
media_channels = mmm.input_data.media_channel.values
media_cols = [f"{channel}_impression" for channel in media_channels]
media_spend_cols = [f"{channel}_spend" for channel in media_channels]
non_media_treatment_cols = ['Promo']
organic_media_cols = ['Organic_channel0_impression']
organic_media_channels = ['Organic_channel0']
revenue_per_kpi_col='revenue_per_conversion'
times_str = [time.strftime(constants.DATE_FORMAT) for time in unique_times]

media_np = np.zeros((n_geos, n_times, n_media_channels))
media_spend_np = np.zeros((n_geos, n_times, n_media_channels))
non_media_treatment_np = np.zeros((n_geos, n_times, n_non_media_channels))
organic_media_np = np.zeros((n_geos, n_times, n_organic_media_channels))
revenue_per_kpi_np = np.zeros((n_geos, n_times))

df_grouped = df.set_index([constants.GEO, constants.TIME])
for geo_idx, geo in enumerate(geos):
  for time_idx, time in enumerate(unique_times):
    row = df_grouped.loc[(geo, time)]
    media_np[geo_idx, time_idx, :] = row[media_cols].values
    media_spend_np[geo_idx, time_idx, :] = row[media_spend_cols].values
    non_media_treatment_np[geo_idx, time_idx, :] = row[non_media_treatment_cols].values
    organic_media_np[geo_idx, time_idx, :] = row[organic_media_cols].values
    revenue_per_kpi_np[geo_idx, time_idx] = row[revenue_per_kpi_col].item()

data_tensors = analyzer.DataTensors(
    media=tf.convert_to_tensor(media_np, dtype=tf.float32),
    media_spend=tf.convert_to_tensor(media_spend_np, dtype=tf.float32),
    non_media_treatments=tf.convert_to_tensor(non_media_treatment_np, dtype=tf.float32),
    organic_media=tf.convert_to_tensor(organic_media_np, dtype=tf.float32),
    revenue_per_kpi=tf.convert_to_tensor(revenue_per_kpi_np, dtype=tf.float32),
    time=tf.convert_to_tensor(times_str, dtype=tf.string),
)
# Default values for `budget` and `pct_of_spend` are derived from the `new_data`,
# but these values can be overridden without modifying the `new_data` itself.
hypothetical_optimization_results = budget_optimizer.optimize(
    new_data=data_tensors,
    budget=50_000_000,
    pct_of_spend=[.2, .1, .2, .2, .3]
)

5\. Export the 2-page HTML optimization report.

In [ ]:
filepath = meridian_root
hypothetical_optimization_results.output_optimization_summary(
    'hypothetical_optimization_output.html', filepath
)

In [ ]:
IPython.display.HTML(filename=f'{meridian_root}/hypothetical_optimization_output.html')

<a name="save-model"></a>
## Step 7: Save the model object

We recommend that you save the model object for future use. This helps you to  avoid repetitive model runs and saves time and computational resources. After the model object is saved, you can load it at a later stage to continue the analysis or visualizations without having to re-run the model.


Run the following codes to save the model object:

In [ ]:
file_path = f'{meridian_root}/saved_mmm.binpb'
meridian_serde.save_meridian(mmm, file_path)
print(f'model is saved at {file_path}')

Run the following codes to load the saved model:

In [ ]:
mmm = meridian_serde.load_meridian(file_path)

In [ ]:
# @markdown If a GCS bucket is mounted, run this cell to unmount it.
!fusermount -u /content/{bucket_name}

<a name="scenario-planning"></a>
## Step 8: Interactive Scenario Planning
[Meridian Scenario Planner](https://developers.google.com/meridian/docs/scenario-planning/meridian-scenario-planner) is a tool that allow advertisers to create a sharable and interactive dashboard from Colab; you can reuse the saved model from this Colab for dashboard generation.